In [1]:
import sys
print(sys.executable)

/usr/bin/python3


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Usa solo la GPU 1

import tensorflow as tf

2025-06-12 17:48:59.910218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-12 17:49:00.542857: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fmartinez/cuda/cudnn/lib64:/usr/local/cuda/lib64:
2025-06-12 17:49:00.542915: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fmartinez/cuda/cudnn/lib64:/usr/local/cuda/lib64:
2025-06-12 17:49:0

In [3]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from pathlib import Path
import os
import random
from plyfile import PlyData, PlyElement
import open3d as o3d
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
import tensorflow as tf
print("Versión de cuDNN:", tf.sysconfig.get_build_info()["cudnn_version"])
print("Versión de CUDA:", tf.sysconfig.get_build_info()["cuda_version"])

Versión de cuDNN: 8
Versión de CUDA: 11.2


In [6]:
import tensorflow as tf

print("Versión de TensorFlow:", tf.__version__)
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))

Versión de TensorFlow: 2.11.0
GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# GOOSE Dataset

In [7]:
import numpy as np
import os
from pathlib import Path
from typing import List, Tuple
from tqdm import tqdm

NUM_POINTS = 4096 # 256, 512, 1024, 2048, 4096, 8192, 16384, 32768

category_mapping = {
    0: [43, 38, 58, 29, 41, 42, 44, 39, 55], # Construction
    1: [4, 45, 6, 40, 60, 61, 33, 32, 14], # Object
    2: [7, 22, 9, 26, 11, 21], # Road
    3: [48, 47, 1, 19, 46, 10, 25], # Sign
    4: [23, 3, 24, 31, 2], # Terrain  
    5: [51, 50, 5, 18], # Drivable Vegetation
    6: [28, 27, 62, 52, 16, 30, 59, 17], # Non Drivable Vegetation
    7: [13, 15, 12, 36, 57, 49, 20, 35, 37, 34, 63], # Vehicle
    8: [8, 56, 0, 53, 54], # Void
}

def filter_points_within_radius(points: np.ndarray, radius: float = 25.0) -> np.ndarray:
    distances = np.linalg.norm(points, axis=1) 
    mask = distances <= radius  
    return points[mask]  

import numpy as np
from typing import Tuple

def load_bin_file(bin_path: str, num_points: int = NUM_POINTS, radius: float = 25.0) -> Tuple[np.ndarray, np.ndarray]:
    # Cargar la nube de puntos
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3] 

    # Filtrar por distancia
    distances = np.linalg.norm(points, axis=1)
    mask = distances <= radius
    points = points[mask]

    num_available = points.shape[0]

    if num_available >= num_points:
        # Seleccionar un punto aleatorio
        center_idx = np.random.randint(0, num_available)

        # Calcular inicio y fin tratando de centrar la ventana
        half_span = num_points // 2
        start_idx = max(0, center_idx - half_span)
        end_idx = start_idx + num_points

        # Si nos pasamos del límite superior, compensar hacia atrás
        if end_idx > num_available:
            end_idx = num_available
            start_idx = max(0, end_idx - num_points)

        # Si nos pasamos del límite inferior, compensar hacia adelante
        if start_idx == 0:
            end_idx = min(num_available, num_points)

        indices = np.arange(start_idx, end_idx)
        return points[indices], np.where(mask)[0][indices]

    else:
        # Si hay menos de num_points disponibles, devolver todo lo que haya
        return points, np.where(mask)[0]


label_to_category = {label: cat for cat, labels in category_mapping.items() for label in labels}

def map_labels(labels: np.ndarray) -> np.ndarray:
    return np.array([label_to_category.get(label, 8) for label in labels], dtype=np.uint8)

def load_label_file(label_path: str, indices: np.ndarray) -> np.ndarray:
    labels = np.fromfile(label_path, dtype=np.uint32) & 0xFFFF 
    return map_labels(labels[indices]) 

def load_dataset(bin_files: List[str], label_files: List[str], num_points: int = NUM_POINTS) -> Tuple[np.ndarray, np.ndarray]:
    x_data, y_data = [], []
    
    for bin_f, label_f in tqdm(zip(bin_files, label_files), total=len(bin_files), desc="Cargando datos"):
        points, indices = load_bin_file(bin_f, num_points)
        
        if points.shape[0] < num_points:
            continue  

        labels = load_label_file(label_f, indices)

        x_data.append(points)
        y_data.append(labels)

    return np.array(x_data, dtype=np.float32), np.array(y_data, dtype=np.uint8)

def get_file_paths(data_dir: str) -> List[str]:
    return sorted([str(f) for f in Path(data_dir).glob("*.*")])

def load_all_data(x_train_dir: str, y_train_dir: str, x_val_dir: str, y_val_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    x_train_files = get_file_paths(x_train_dir)
    y_train_files = get_file_paths(y_train_dir)
    x_val_files = get_file_paths(x_val_dir)
    y_val_files = get_file_paths(y_val_dir)
    
    assert len(x_train_files) == len(y_train_files), "Número de archivos x_train y y_train no coincide."
    assert len(x_val_files) == len(y_val_files), "Número de archivos x_val y y_val no coincide."
    
    print("Cargando datos de entrenamiento...")
    x_train, y_train = load_dataset(x_train_files, y_train_files)
    
    print("Cargando datos de validación...")
    x_val, y_val = load_dataset(x_val_files, y_val_files)
    
    return x_train, y_train, x_val, y_val

In [10]:
import numpy as np
import random
from tensorflow.keras.utils import Sequence
from scipy.spatial import cKDTree

class PointCloudGenerator(Sequence):
    def __init__(self, bin_files, label_files, num_points, mode='train', batch_size=32):
        self.bin_files = bin_files
        self.label_files = label_files
        self.num_points = num_points
        self.batch_size = batch_size
        self.mode = mode  # 'train' o 'val'
        self.on_epoch_end()

    def __len__(self):
        """Número total de batches por epoch."""
        return len(self.bin_files) // self.batch_size

    def __getitem__(self, idx):
        """Genera un batch de datos."""
        batch_bin_files = self.bin_files[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_label_files = self.label_files[idx * self.batch_size : (idx + 1) * self.batch_size]

        x_batch, y_batch = [], []

        for bin_file, label_file in zip(batch_bin_files, batch_label_files):
            points, indices = load_bin_file(bin_file, self.num_points)
            labels = load_label_file(label_file, indices)

            if self.mode == 'train':
                sub_points, sub_labels = self._get_train_sample(points, labels)
                x_batch.append(sub_points)
                y_batch.append(sub_labels)
            else:
                sub_points_list, sub_labels_list = self._get_val_samples(points, labels)
                x_batch.extend(sub_points_list)  # Agrega cada subnube individualmente
                y_batch.extend(sub_labels_list)

        return np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.uint8)

    def on_epoch_end(self):
        """Mezcla los datos al final de cada epoch."""
        temp = list(zip(self.bin_files, self.label_files))
        random.shuffle(temp)
        self.bin_files, self.label_files = zip(*temp)

    def _get_train_sample(self, points, labels):
        """Obtiene una subnube de entrenamiento con N puntos."""
        center_idx = np.random.randint(len(points))
        tree = cKDTree(points)
        _, neighbor_indices = tree.query(points[center_idx], k=self.num_points)
        return points[neighbor_indices], labels[neighbor_indices]

    def _get_val_samples(self, points, labels):
        """Divide la nube en subnubes de N puntos para validación."""
        num_full_batches = len(points) // self.num_points
        indices = np.random.choice(len(points), num_full_batches * self.num_points, replace=False)
        subclouds = points[indices].reshape(num_full_batches, self.num_points, 3)
        sublabels = labels[indices].reshape(num_full_batches, self.num_points)
        
        # Convertir de (num_subclouds, num_points, 3) a lista de subnubes individuales
        return [subclouds[i] for i in range(num_full_batches)], [sublabels[i] for i in range(num_full_batches)]


In [11]:
x_train_path = "/home/fmartinez/datasets/goose/lidar/train"
y_train_path = "/home/fmartinez/datasets/goose/labels/train"

x_val_path = "/home/fmartinez/datasets/goose/lidar/val"
y_val_path = "/home/fmartinez/datasets/goose/labels/val"

x_test_path = "/home/fmartinez/datasets_val/lidar/val"
y_test_path = "/home/fmartinez/datasets_val/labels/val"

x_train_files = get_file_paths(x_train_path)
y_train_files = get_file_paths(y_train_path)
x_val_files = get_file_paths(x_val_path)
y_val_files = get_file_paths(y_val_path)
x_test_files = get_file_paths(x_test_path)
y_test_files = get_file_paths(y_test_path)

In [12]:
train_generator = PointCloudGenerator(x_train_files, y_train_files, num_points=16384, mode='train', batch_size=16)
val_generator = PointCloudGenerator(x_val_files, y_val_files, num_points=16384, mode='val', batch_size=16)
test_generator = PointCloudGenerator(x_test_files, y_test_files, num_points=16384, mode='val', batch_size=16)

In [13]:
class_weights = {0: 1.3756800728474192, 1: 11.540744634253176, 2: 14.010762980452215, 3: 18.564527493588116, 4: 1.0714398416315722, 5: 0.4806517192715152, 6: 0.20640943202058679, 7: 6.540442855724357, 8: 20.108312727050706}


In [19]:
import tensorflow as tf
from tensorflow.keras import layers, Model

MAX_POINTS = 16384

def tnet(inputs, num_features):
    x = layers.Conv1D(64, 1, activation='relu', padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(1024, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(num_features * num_features, kernel_initializer='zeros',
                     bias_initializer=tf.keras.initializers.Constant(tf.eye(num_features).numpy().flatten()))(x)
    transform_matrix = layers.Reshape((num_features, num_features))(x)

    def transform(inputs_and_matrix):
        inputs, matrix = inputs_and_matrix
        return tf.matmul(inputs, matrix)

    transformed_inputs = layers.Lambda(transform)([inputs, transform_matrix])
    transformed_inputs = layers.Lambda(lambda t: tf.ensure_shape(t, (None, MAX_POINTS, num_features)))(transformed_inputs)

    return transformed_inputs

def build_pointnet(num_classes, input_dim=3, max_points=MAX_POINTS):
    inputs = tf.keras.Input(shape=(None, input_dim))

    x = tnet(inputs, input_dim)

    x = layers.Conv1D(64, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = tnet(x, 64)

    x = layers.Conv1D(1024, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    global_features = layers.GlobalMaxPooling1D()(x)
    global_features = layers.Lambda(lambda t: tf.expand_dims(t, axis=1))(global_features)
    global_features = layers.Lambda(lambda t: tf.repeat(t, repeats=max_points, axis=1))(global_features)

    x = layers.Lambda(lambda t: tf.ensure_shape(t, (None, max_points, 1024)))(x)

    x = layers.concatenate([x, global_features], axis=-1)

    x = layers.Conv1D(512, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(256, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Conv1D(num_classes, 1, activation='softmax')(x)

    return Model(inputs, outputs)

In [20]:
import tensorflow as tf

class MeanIoUWrapper(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou_wrapper", **kwargs):
        super(MeanIoUWrapper, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(y_pred, axis=-1)  # Convertir (batch, 16384, 9) -> (batch, 16384)
        self.metric.update_state(y_true, y_pred_labels)

    def result(self):
        return self.metric.result()

    def reset_state(self):
        self.metric.reset_state()

class MeanIoUWrapper_2(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou_wrapper", **kwargs):
        super(MeanIoUWrapper, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32)  # Asegurar que y_true sea int32
        y_pred_labels = tf.argmax(y_pred, axis=-1)  # Convertir (batch, 16384, 9) -> (batch, 16384)

        self.metric.update_state(y_true, y_pred_labels, sample_weight)  # Pasar sample_weight

    def result(self):
        return self.metric.result()

    def reset_state(self):
        self.metric.reset_state()


In [21]:
import tensorflow as tf

# Convertir los pesos a tensores
class_weight_tensor = tf.constant([class_weights[i] for i in range(9)], dtype=tf.float32)

def weighted_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)
    sample_weights = tf.gather(class_weight_tensor, y_true)  # Asigna el peso según la clase
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss * sample_weights  # Escala la pérdida por el peso de la clase

In [22]:
INPUT_DIM = 3
NUM_CLASSES = 9

# Definir el modelo PointNet
model = build_pointnet(num_classes=NUM_CLASSES, input_dim=INPUT_DIM)
mean_iou_wrapper = MeanIoUWrapper(num_classes=NUM_CLASSES)
optimizer = tf.optimizers.Adam(learning_rate=0.001)

# Compilar el modelo
model.compile(
    optimizer = optimizer,
    loss=weighted_loss,
    metrics=["accuracy", mean_iou_wrapper]
)

In [23]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    verbose=1
)

Epoch 1/30
 75/422 [====>.........................] - ETA: 3:50 - loss: 2.5194 - accuracy: 0.1591 - mean_iou_wrapper: 0.0537

2025-06-12 17:54:16.648515: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: IndexError: index 11446 is out of bounds for axis 0 with size 11446
Traceback (most recent call last):

  File "/home/fmartinez/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/fmartinez/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/fmartinez/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/fmartinez/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "/home/fmartinez/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 1048, in generator_fn
    yield x[i]

  File "/tmp/ipykernel_1113031

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  IndexError: index 11446 is out of bounds for axis 0 with size 11446
Traceback (most recent call last):

  File "/home/fmartinez/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/fmartinez/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/fmartinez/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/fmartinez/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "/home/fmartinez/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 1048, in generator_fn
    yield x[i]

  File "/tmp/ipykernel_1113031/332293

In [20]:
model.save("pointnet_goose_16k_3.keras")

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Supongamos que estos vienen de tu modelo entrenado (Keras history)
loss = history.history['loss']
val_loss = history.history.get('val_loss')
mean_iou = history.history.get('mean_iou_wrapper')
val_mean_iou = history.history.get('val_mean_iou_wrapper')
accuracy = history.history.get('accuracy')
val_accuracy = history.history.get('val_accuracy')

# Crear figura con 1 fila y 3 columnas de subplots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Loss Curve", "Mean IoU Curve", "Accuracy Curve")
)

# 1) GRÁFICO DE PÉRDIDA (col=1)
epochs_loss = list(range(1, len(loss) + 1))

fig.add_trace(
    go.Scatter(
        x=epochs_loss,
        y=loss,
        mode='lines+markers',
        name='Train Loss'
    ),
    row=1, col=1
)

if val_loss:
    epochs_val_loss = list(range(1, len(val_loss) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_val_loss,
            y=val_loss,
            mode='lines+markers',
            name='Validation Loss'
        ),
        row=1, col=1
    )

fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1, range=[0, 10])  # Rango ajustado

# 2) GRÁFICO DE Mean IoU (col=2) -> Limitar valores a [0,1]
if mean_iou:
    mean_iou = np.clip(mean_iou, 0, 1)
    epochs_mean_iou = list(range(1, len(mean_iou) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_mean_iou,
            y=mean_iou,
            mode='lines+markers',
            name='Train Mean IoU'
        ),
        row=1, col=2
    )

    if val_mean_iou:
        val_mean_iou = np.clip(val_mean_iou, 0, 1)
        epochs_val_mean_iou = list(range(1, len(val_mean_iou) + 1))
        fig.add_trace(
            go.Scatter(
                x=epochs_val_mean_iou,
                y=val_mean_iou,
                mode='lines+markers',
                name='Validation Mean IoU'
            ),
            row=1, col=2
        )

fig.update_xaxes(title_text='Epochs', row=1, col=2)
fig.update_yaxes(title_text='Mean IoU', row=1, col=2, range=[0, 1])

# 3) GRÁFICO DE ACCURACY (col=3) -> Limitar valores a [0,1]
if accuracy:
    accuracy = np.clip(accuracy, 0, 1)
    epochs_acc = list(range(1, len(accuracy) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_acc,
            y=accuracy,
            mode='lines+markers',
            name='Train Accuracy'
        ),
        row=1, col=3
    )

    if val_accuracy:
        val_accuracy = np.clip(val_accuracy, 0, 1)
        epochs_val_acc = list(range(1, len(val_accuracy) + 1))
        fig.add_trace(
            go.Scatter(
                x=epochs_val_acc,
                y=val_accuracy,
                mode='lines+markers',
                name='Validation Accuracy'
            ),
            row=1, col=3
        )

fig.update_xaxes(title_text='Epochs', row=1, col=3)
fig.update_yaxes(title_text='Accuracy', row=1, col=3, range=[0, 1])

# Ajustes generales de la figura
fig.update_layout(
    width=1300,
    height=500,
    showlegend=True
)

fig.show()